In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

from hotutils import areamean_utils as am
from hotutils import scplot_utils as scplt
from hotutils import season_utils as sc

import warnings
warnings.filterwarnings('ignore')
plt.rc('font', family='Arial')

### Read files

In [2]:
#read files
path1      = '/home/donghq/dong_hq/cmip6/heatwave/hist_ssp/tasmax/MME/'
path2      = '/home/donghq/dong_hq/cmip6/LMIP-pdLC/tmax/MME/'
path3      = '/home/donghq/dong_hq/cmip6/heatwave/hist_ssp/pr/monthly/MME/'
path4      = '/home/donghq/dong_hq/cmip6/LMIP-pdLC/pr/MME/'
time_his   = slice('1986','2005')
time_ssp   = slice('2080','2099')
#tasmax
tmax_cm    = xr.open_dataset(path1 + 'tasmax_Amon_5MME_historical_ssp_19790101-21001231_regrid.nc').tasmax
tmax_lm    = xr.open_dataset(path2 + 'tasmax_Amon_5MME_r1i1p1f2_amip-lfmip-pdLC_198001-210012_regrid.nc').tasmax
pr_cm      = xr.open_dataset(path3 + 'pr_Amon_6MME_historcial_ssp585_r1i1p1f1_196201-209912_regrid.nc').pr * 24 * 3600
pr_lm      = xr.open_dataset(path4 + 'pr_Amon_6MME_amip-lfmip-pdLC_r1i1p1f2_198001-209912_regrid.nc').pr * 24 * 3600

### Calculate area-mean

In [3]:
#由于 CESM2 LFMIP数据仅有陆地格点，对此单独分析
nlat_range = slice(0,25)
slat_range = slice(-25,0)
def cesm2_lmip(ds,lat_range):
    ds_am0 = am.ram(ds[0].sel(lat=lat_range))
    ds_am1 = am.ram(ds[1:].sel(lat=lat_range),'land')
    ds_am  = xr.concat([ds_am0,ds_am1],dim='model')
    return ds_am.transpose('model','time')

In [4]:
#NHL
#hist
prc_nh_amh = am.ram(pr_cm.sel(time=time_his,lat=slice(0,25)),'land')
prl_nh_amh = cesm2_lmip(pr_lm.sel(time=time_his),nlat_range)
tmc_nh_amh = am.ram(tmax_cm.sel(time=time_his,lat=slice(0,25)),'land')
tml_nh_amh = am.ram(tmax_lm.sel(time=time_his,lat=slice(0,25)),'land')
#ssp
prc_nh_ams = am.ram(pr_cm.sel(time=time_ssp,lat=slice(0,25)),'land')
prl_nh_ams = cesm2_lmip(pr_lm.sel(time=time_ssp),nlat_range)
tmc_nh_ams = am.ram(tmax_cm.sel(time=time_ssp,lat=slice(0,25)),'land')
tml_nh_ams = am.ram(tmax_lm.sel(time=time_ssp,lat=slice(0,25)),'land')
#SHL
#hist
prc_sh_amh = am.ram(pr_cm.sel(time=time_his,lat=slice(-25,0)),'land')
prl_sh_amh = cesm2_lmip(pr_lm.sel(time=time_his),slat_range)
tmc_sh_amh = am.ram(tmax_cm.sel(time=time_his,lat=slice(-25,0)),'land')
tml_sh_amh = am.ram(tmax_lm.sel(time=time_his,lat=slice(-25,0)),'land')
#ssp
prc_sh_ams = am.ram(pr_cm.sel(time=time_ssp,lat=slice(-25,0)),'land')
prl_sh_ams = cesm2_lmip(pr_lm.sel(time=time_ssp),slat_range)
tmc_sh_ams = am.ram(tmax_cm.sel(time=time_ssp,lat=slice(-25,0)),'land')
tml_sh_ams = am.ram(tmax_lm.sel(time=time_ssp,lat=slice(-25,0)),'land')

### Calculate seasonal cycl

In [5]:
#NHL
dpc_nh_sc,dpc_nh_p,dpc_nh_tm,dpc_nh_hc = sc.dps(prc_nh_ams.mean('model'),prc_nh_amh.mean('model'))
dpl_nh_sc,dpl_nh_p,dpl_nh_tm,dpl_nh_hc = sc.dps(prl_nh_ams.mean('model'),prl_nh_amh.mean('model'))
dmc_nh_sc,dmc_nh_p,dmc_nh_tm,dmc_nh_hc = sc.dps(tmc_nh_ams.mean('model'),tmc_nh_amh.mean('model'))
dml_nh_sc,dml_nh_p,dml_nh_tm,dml_nh_hc = sc.dps(tml_nh_ams.mean('model'),tml_nh_amh.mean('model'))
ddp_nh_tdm,ddp_nh_p,ddp_nh_tm,_        = sc.dps(prc_nh_ams.mean('model') - prl_nh_ams.mean('model'), prc_nh_amh.mean('model') - prl_nh_amh.mean('model'))
ddp_sh_tdm,ddp_sh_p,ddp_sh_tm,_        = sc.dps(prc_sh_ams.mean('model') - prl_sh_ams.mean('model'), prc_sh_amh.mean('model') - prl_sh_amh.mean('model'))
prc_nh_MM                              = sc.dps_err(prc_nh_ams,prc_nh_amh)
prl_nh_MM                              = sc.dps_err(prl_nh_ams,prl_nh_amh)
tmc_nh_MM                              = sc.dps_err(tmc_nh_ams,tmc_nh_amh)
tml_nh_MM                              = sc.dps_err(tml_nh_ams,tml_nh_amh)
dpr_nh_MM                              = sc.dps_err_MM(prc_nh_ams,prl_nh_ams,prc_nh_amh,prl_nh_amh)
dtm_nh_MM                              = sc.dps_err_MM(tmc_nh_ams,tml_nh_ams,tmc_nh_amh,tml_nh_amh)
#SHL
dpc_sh_sc,dpc_sh_p,dpc_sh_tm,dpc_sh_hc = sc.dps(prc_sh_ams.mean('model'),prc_sh_amh.mean('model'))
dpl_sh_sc,dpl_sh_p,dpl_sh_tm,dpl_sh_hc = sc.dps(prl_sh_ams.mean('model'),prl_sh_amh.mean('model'))
dmc_sh_sc,dmc_sh_p,dmc_sh_tm,dmc_sh_hc = sc.dps(tmc_sh_ams.mean('model'),tmc_sh_amh.mean('model'))
dml_sh_sc,dml_sh_p,dml_sh_tm,dml_sh_hc = sc.dps(tml_sh_ams.mean('model'),tml_sh_amh.mean('model'))
ddm_nh_tdm,ddm_nh_p,ddm_nh_tm,_        = sc.dps(tmc_nh_ams.mean('model') - tml_nh_ams.mean('model'), tmc_nh_amh.mean('model') - tml_nh_amh.mean('model'))
ddm_sh_tdm,ddm_sh_p,ddm_sh_tm,_        = sc.dps(tmc_sh_ams.mean('model') - tml_sh_ams.mean('model'), tmc_sh_amh.mean('model') - tml_sh_amh.mean('model'))
prc_sh_MM                              = sc.dps_err(prc_sh_ams,prc_sh_amh)
prl_sh_MM                              = sc.dps_err(prl_sh_ams,prl_sh_amh)
tmc_sh_MM                              = sc.dps_err(tmc_sh_ams,tmc_sh_amh)
tml_sh_MM                              = sc.dps_err(tml_sh_ams,tml_sh_amh)
dpr_sh_MM                              = sc.dps_err_MM(prc_sh_ams,prl_sh_ams,prc_sh_amh,prl_sh_amh)
dtm_sh_MM                              = sc.dps_err_MM(tmc_sh_ams,tml_sh_ams,tmc_sh_amh,tml_sh_amh)

### Calculate the season contribute rate of Soil Moisture

In [6]:
#TMAX
#AMJ (SON)
cte_nh_amj = ddm_nh_tdm[3:6].mean()/dmc_nh_sc[3:6].mean()
cte_sh_son = ddm_sh_tdm[8:11].mean()/dmc_sh_sc[8:11].mean()
# Pr
ctp_nh_amj = ddp_nh_tdm[3:6].mean()/dpc_nh_sc[3:6].mean()
ctp_sh_son = ddp_sh_tdm[8:11].mean()/dpc_sh_sc[8:11].mean()

### Plot

In [7]:
ft = 11
fig = plt.figure(figsize=(22, 18), dpi=500)
plt.rcParams['hatch.color'] = 'white'
plt.rcParams['hatch.linewidth'] = 0.7
scplt.cm_scn(fig,dpc_nh_sc,dpc_nh_hc,dpc_nh_p,prc_nh_MM,0.9,[0,6],0.3,1,[-1.1,0.9*8/7],'a','PR','Fully coupled','NTL (0-25N)','Change (mm d$^{-1}$)','Climatology (mm d$^{-1}$)',ft,4,3,1,if_legend=True)
scplt.cm_scn(fig,dpl_nh_sc,dpl_nh_hc,dpl_nh_p,prl_nh_MM,0.9,[0,6],0.3,1,[-1.1,0.9*8/7],'b','PR','Fixed SM','NTL (0-25N)','Change (mm d$^{-1}$)','Climatology (mm d$^{-1}$)',ft,4,3,2,if_legend=False)
scplt.sa_scn(fig,ddp_nh_tdm,ddp_nh_p,dpr_nh_MM,ctp_nh_amj,5,-0.45,0.6,0.2,[-1.1,0.6*8/7],'c','PR','SA','NTL (0-25N)','Change (mm d$^{-1}$)',ft,4,3,3,if_legend=False,direct='out',if_cte=True)
scplt.cm_scs(fig,dpc_sh_sc,dpc_sh_hc,dpc_sh_p,prc_sh_MM,1.2,[0,8],0.4,1,[-1.1,1.2*8/7],'d','PR','Fully coupled','STL (0-25S)','Change (mm d$^{-1}$)','Climatology (mm d$^{-1}$)',ft,4,3,4,if_legend=False)
scplt.cm_scs(fig,dpl_sh_sc,dpl_sh_hc,dpl_sh_p,prl_sh_MM,1.2,[0,8],0.4,1,[-1.1,1.2*8/7],'e','PR','Fixed SM','STL (0-25S)','Change (mm d$^{-1}$)','Climatology (mm d$^{-1}$)',ft,4,3,5,if_legend=False)
scplt.sa_scs(fig,ddp_sh_tdm,ddp_sh_p,dpr_sh_MM,ctp_sh_son,4,-0.5,0.6,0.2,[-1.1,0.6*8/7],'f','PR','SA','STL (0-25S)','Change (mm d$^{-1}$)',ft,4,3,6,if_legend=False,direct='out',if_cte=True)
scplt.cm_scn(fig,dmc_nh_sc,dmc_nh_hc,dmc_nh_p,tmc_nh_MM,0.9,[299,307],0.3,1,[-1.1,0.9*8/7],'g','TX','Fully coupled','NTL (0-25N)','Change (K)','Climatology (K)',ft,4,3,7,if_var='tx',if_legend=True)
scplt.cm_scn(fig,dml_nh_sc,dml_nh_hc,dml_nh_p,tml_nh_MM,0.9,[299,307],0.3,1,[-1.1,0.9*8/7],'h','TX','Fixed SM','NTL (0-25N)','Change (K)','Climatology (K)',ft,4,3,8,if_var='tx',if_legend=False)
scplt.sa_scn(fig,ddm_nh_tdm,ddm_nh_p,dtm_nh_MM,cte_nh_amj,5,-0.45,0.6,0.2,[-1.1,0.6*8/7],'i','TX','SA','NTL (0-25N)','Change (K)',ft,4,3,9,if_legend=False,if_cte=True)
scplt.cm_scs(fig,dmc_sh_sc,dmc_sh_hc,dmc_sh_p,tmc_sh_MM,1.5,[297,305],0.5,1,[-1.1,1.5*8/7],'j','TX','Fully coupled','STL (0-25S)','Change (K)','Climatology (K)',ft,4,3,10,if_var='tx',if_legend=False)
scplt.cm_scs(fig,dml_sh_sc,dml_sh_hc,dml_sh_p,tml_sh_MM,1.5,[297,305],0.5,1,[-1.1,1.5*8/7],'k','TX','Fixed SM','STL (0-25S)','Change (K)','Climatology (K)',ft,4,3,11,if_var='tx',if_legend=False)
scplt.sa_scs(fig,ddm_sh_tdm,ddm_sh_p,dtm_sh_MM,cte_sh_son,4,-0.9,1.2,0.4,[-1.1,1.2*8/7],'l','TX','SA','STL (0-25S)','Change (K)',ft,4,3,12,if_legend=False,if_cte=True)
plt.subplots_adjust(wspace=0.3,hspace=0.3)
plt.show()